# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,41.45,56,0,18.41,CL,1715637656
1,1,sao vicente,-23.9631,-46.3919,91.58,43,0,3.44,BR,1715637657
2,2,albany,42.6001,-73.9662,64.81,65,100,2.98,US,1715637362
3,3,ushuaia,-54.8000,-68.3000,40.66,70,40,36.82,AR,1715637657
4,4,hermanus,-34.4187,19.2345,57.79,82,53,10.78,ZA,1715637657


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
humidity_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size="Humidity",
    geo=True,
    frame_width=800,
    frame_height=600,
    hover_cols=["City"], 
    xlim=(-180, 180),  
    ylim=(-90, 90),
    tiles='CartoLight',
    color = "City"
)

# Display the map
humidity_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df[
    (city_data_df['Max Temp'] >= 75) &
    (city_data_df['Max Temp'] <= 80) &
    (city_data_df['Humidity'] >= 30) &
    (city_data_df['Humidity'] <= 50)
]
# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
ideal_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
151,151,fort bragg,35.1390,-79.0060,78.82,45,100,8.05,US,1715637667
350,350,mandan,46.8267,-100.8896,77.81,32,0,11.50,US,1715637685
438,438,sharm el-sheikh,27.8518,34.3050,78.94,47,0,11.50,EG,1715637649
448,448,tripoli,32.8752,13.1875,79.43,33,2,12.82,LY,1715637694
464,464,marsa alam,25.0757,34.8918,76.21,43,0,11.56,EG,1715637696


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotels_df to store the city, country, coordinates, and humidity
hotel_columns = ["City", "Country", "Lat", "Lng", "Humidity"]
hotels_df = ideal_weather[hotel_columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame to store the hotel found using the Geoapify API
hotels_df['Hotel'] = pd.Series(dtype='object')

# Display sample data
hotels_df.head()


,City,Country,Lat,Lng,Humidity,Hotel
151,fort bragg,US,35.1390,-79.0060,45,NaN
350,mandan,US,46.8267,-100.8896,32,NaN
438,sharm el-sheikh,EG,27.8518,34.3050,47,NaN
448,tripoli,LY,32.8752,13.1875,33,NaN
464,marsa alam,EG,25.0757,34.8918,43,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000

params = {
    "categories": categories,
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in city_data_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    name_address = response.json()
 
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_name = name_address["features"][0]["properties"]["name"]
        # Assign value using .loc[] to avoid the warning
        city_data_df.loc[index, "Hotel"] = hotel_name
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        city_data_df.loc[index, "Hotel"] = "No hotel found"
        
    # Log the search results
    print(f"{city_data_df.loc[index, 'City']} - nearest hotel: {city_data_df.loc[index, 'Hotel']}")

# Display sample data
city_data_df


Starting hotel search
puerto natales - nearest hotel: Dorotea Patagonia Hostel
sao vicente - nearest hotel: Pousada Vitória
albany - nearest hotel: No hotel found
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
hermanus - nearest hotel: Aloe guest house
edinburgh of the seven seas - nearest hotel: No hotel found
torghay - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
blackmans bay - nearest hotel: Villa Howden
yaren - nearest hotel: Aiwo Hotel
longyearbyen - nearest hotel: The Vault
balabac - nearest hotel: No hotel found
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
lages - nearest hotel: Hotel Ideal
nemuro - nearest hotel: お宿エクハシ
waitangi - nearest hotel: Hotel Chathams
nar'yan-mar - nearest hotel: Гостиница "Агат"
susuman - nearest hotel: No hotel found
ar rutbah - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
mayqayyng - nearest hotel: No hotel found
n

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
0,0,puerto natales,-51.7236,-72.4875,41.45,56,0,18.41,CL,1715637656,Dorotea Patagonia Hostel
1,1,sao vicente,-23.9631,-46.3919,91.58,43,0,3.44,BR,1715637657,Pousada Vitória
2,2,albany,42.6001,-73.9662,64.81,65,100,2.98,US,1715637362,No hotel found
3,3,ushuaia,-54.8000,-68.3000,40.66,70,40,36.82,AR,1715637657,Apart Hotel Aires del Beagle
4,4,hermanus,-34.4187,19.2345,57.79,82,53,10.78,ZA,1715637657,Aloe guest house
...,...,...,...,...,...,...,...,...,...,...,...
584,584,kununurra,-15.7667,128.7333,69.78,73,33,5.75,AU,1715637705,No hotel found
585,585,port saint john's,-31.6229,29.5448,68.09,88,91,8.30,ZA,1715637706,Outback Inn
586,586,weno,7.4515,151.8468,83.73,82,100,15.82,FM,1715637706,High Tide Hotel
587,587,howard springs,-12.4970,131.0470,78.71,85,2,6.08,AU,1715637706,Rydges Palmerston


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
hotels_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Hotels",
    geo=True,
    frame_width=800,
    frame_height=600,
    hover_cols=["City", "Country", "Hotel"], 
    xlim=(-180, 180),  
    ylim=(-90, 90),
    tiles='CartoLight',
    color = "City"
)

# Display the map
hotels_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel)